As seguintes bibliotecas adicionais são necessárias para executar este
notebook. Observe que a execução no Colab é experimental, por favor, relate um Github
questão se você tiver algum problema.


In [ ]:
!pip install d2l==1.0.3


# Inferência de linguagem natural: ajuste fino de BERT
:label:`sec_natural-language-inference-bert`

Nas seções anteriores deste capítulo,
nós projetamos uma arquitetura baseada na atenção
(em :numref:`sec_natural-language-inference-attention`)
para a tarefa de inferência de linguagem natural
no conjunto de dados SNLI (conforme descrito em :numref:`sec_natural-language-inference-and-dataset`).
Agora revisitamos essa tarefa ajustando o BERT.
Conforme discutido em :numref:`sec_finetuning-bert`,
a inferência em linguagem natural é um problema de classificação de pares de texto em nível de sequência,
e o ajuste fino do BERT requer apenas uma arquitetura adicional baseada em MLP,
conforme ilustrado em :numref:`fig_nlp-map-nli-bert`.

![Esta seção alimenta BERT pré-treinado para uma arquitetura baseada em MLP para inferência de linguagem natural.](https://github.com/d2l-ai/d2l-pytorch-colab/blob/master/img/nlp-map-nli-bert.svg?raw=1)
:label:`fig_nlp-map-nli-bert`

Nesta seção,
faremos o download de uma versão pequena pré-treinada do BERT,
então ajuste-o
para inferência de linguagem natural no conjunto de dados SNLI.


In [1]:
import json
import multiprocessing
import os
import torch
from torch import nn
from d2l import torch as d2l

## [**Carregando BERT pré-treinado**]

Explicamos como pré-treinar BERT no conjunto de dados WikiText-2 em
:numref:`sec_bert-dataset` e :numref:`sec_bert-pretraining`
(observe que o modelo BERT original é pré-treinado em corpora muito maiores).
Conforme discutido em :numref:`sec_bert-pretraining`,
o modelo BERT original tem centenas de milhões de parâmetros.
A seguir,
Nós fornecemos duas versões de BERT pré-treinado:
"bert.base" é quase tão grande quanto o modelo base BERT original que requer muitos recursos computacionais para ajuste fino,
enquanto "bert.small" é uma versão pequena para facilitar a demonstração.


In [2]:
d2l.DATA_HUB['bert.base'] = (d2l.DATA_URL + 'bert.base.torch.zip',
                             '225d66f04cae318b841a13d32af3acc165f253ac')
d2l.DATA_HUB['bert.small'] = (d2l.DATA_URL + 'bert.small.torch.zip',
                              'c72329e68a732bef0452e4b96a1c341c8910f81f')

Qualquer modelo BERT pré-treinado contém um arquivo "vocab.json" que define o conjunto de vocabulário
e um arquivo "pretrained.params" dos parâmetros pré-treinados.
Implementamos a seguinte função `load_pretrained_model` para [**carregar parâmetros BERT pré-treinados**].


In [3]:
def load_pretrained_model(pretrained_model, num_hiddens, ffn_num_hiddens,
                          num_heads, num_blks, dropout, max_len, devices):
    data_dir = d2l.download_extract(pretrained_model)
    # Define an empty vocabulary to load the predefined vocabulary
    vocab = d2l.Vocab()
    vocab.idx_to_token = json.load(open(os.path.join(data_dir, 'vocab.json')))
    vocab.token_to_idx = {token: idx for idx, token in enumerate(
        vocab.idx_to_token)}
    bert = d2l.BERTModel(
        len(vocab), num_hiddens, ffn_num_hiddens=ffn_num_hiddens, num_heads=4,
        num_blks=2, dropout=0.2, max_len=max_len)
    # Load pretrained BERT parameters
    bert.load_state_dict(torch.load(os.path.join(data_dir,
                                                 'pretrained.params')))
    return bert, vocab

Para facilitar a demonstração na maioria das máquinas,
Nesta seção, carregaremos e ajustaremos a versão pequena ("bert.small") do BERT pré-treinado.
No exercício, mostraremos como ajustar o "bert.base" muito maior para melhorar significativamente a precisão dos testes.


In [4]:
devices = d2l.try_all_gpus()
bert, vocab = load_pretrained_model(
    'bert.small', num_hiddens=256, ffn_num_hiddens=512, num_heads=4,
    num_blks=2, dropout=0.1, max_len=512, devices=devices)

/tmp/ipykernel_196239/325226183.py:13: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  bert.load_state_dict(torch.load(os.path.join(data_dir,


## [**O conjunto de dados para ajuste fino do BERT**]

Para a tarefa de downstream inferência de linguagem natural no conjunto de dados SNLI,
definimos uma classe de conjunto de dados personalizada `SNLIBERTDataset`.
Em cada exemplo,
a premissa e a hipótese formam um par de sequências de texto
e é compactado em uma sequência de entrada BERT, conforme descrito em :numref:`fig_bert-two-seqs`.
Lembre-se de :numref:`subsec_bert_input_rep` que os IDs de segmento
são usados ​​para distinguir a premissa e a hipótese em uma sequência de entrada BERT.
Com o comprimento máximo predefinido de uma sequência de entrada BERT (`max_len`),
o último token do par de texto de entrada mais longo continua sendo removido até
`max_len` é atendido.
Para acelerar a geração do conjunto de dados SNLI
para ajuste fino do BERT,
Usamos 4 processos de trabalho para gerar exemplos de treinamento ou teste em paralelo.


In [5]:
class SNLIBERTDataset(torch.utils.data.Dataset):
    def __init__(self, dataset, max_len, vocab=None):
        all_premise_hypothesis_tokens = [[
            p_tokens, h_tokens] for p_tokens, h_tokens in zip(
            *[d2l.tokenize([s.lower() for s in sentences])
              for sentences in dataset[:2]])]

        self.labels = torch.tensor(dataset[2])
        self.vocab = vocab
        self.max_len = max_len
        (self.all_token_ids, self.all_segments,
         self.valid_lens) = self._preprocess(all_premise_hypothesis_tokens)
        print('read ' + str(len(self.all_token_ids)) + ' examples')

    def _preprocess(self, all_premise_hypothesis_tokens):
        pool = multiprocessing.Pool(4)  # Use 4 worker processes
        out = pool.map(self._mp_worker, all_premise_hypothesis_tokens)
        all_token_ids = [
            token_ids for token_ids, segments, valid_len in out]
        all_segments = [segments for token_ids, segments, valid_len in out]
        valid_lens = [valid_len for token_ids, segments, valid_len in out]
        return (torch.tensor(all_token_ids, dtype=torch.long),
                torch.tensor(all_segments, dtype=torch.long),
                torch.tensor(valid_lens))

    def _mp_worker(self, premise_hypothesis_tokens):
        p_tokens, h_tokens = premise_hypothesis_tokens
        self._truncate_pair_of_tokens(p_tokens, h_tokens)
        tokens, segments = d2l.get_tokens_and_segments(p_tokens, h_tokens)
        token_ids = self.vocab[tokens] + [self.vocab['<pad>']] \
                             * (self.max_len - len(tokens))
        segments = segments + [0] * (self.max_len - len(segments))
        valid_len = len(tokens)
        return token_ids, segments, valid_len

    def _truncate_pair_of_tokens(self, p_tokens, h_tokens):
        # Reserve slots for '<CLS>', '<SEP>', and '<SEP>' tokens for the BERT
        # input
        while len(p_tokens) + len(h_tokens) > self.max_len - 3:
            if len(p_tokens) > len(h_tokens):
                p_tokens.pop()
            else:
                h_tokens.pop()

    def __getitem__(self, idx):
        return (self.all_token_ids[idx], self.all_segments[idx],
                self.valid_lens[idx]), self.labels[idx]

    def __len__(self):
        return len(self.all_token_ids)

Após baixar o conjunto de dados SNLI,
nós [**geramos exemplos de treinamento e teste**]
instanciando a classe `SNLIBERTDataset`.
Esses exemplos serão lidos em minibatches durante o treinamento e os testes
de inferência de linguagem natural.


In [ ]:
# Reduce `batch_size` if there is an out of memory error. In the original BERT
# model, `max_len` = 512
batch_size, max_len, num_workers = 512, 128, d2l.get_dataloader_workers()
data_dir = d2l.download_extract('SNLI')
train_set = SNLIBERTDataset(d2l.read_snli(data_dir, True), max_len, vocab)
test_set = SNLIBERTDataset(d2l.read_snli(data_dir, False), max_len, vocab)
train_iter = torch.utils.data.DataLoader(train_set, batch_size, shuffle=True,
                                   num_workers=num_workers)
test_iter = torch.utils.data.DataLoader(test_set, batch_size,
                                  num_workers=num_workers)

read 9824 examples


## Ajuste fino do BERT

Como :numref:`fig_bert-two-seqs` indica,
ajuste fino de BERT para inferência de linguagem natural
requer apenas um MLP extra consistindo de duas camadas totalmente conectadas
(veja `self.hidden` e `self.output` na seguinte classe `BERTClassifier`).
[**Este MLP transforma o
Representação BERT do token especial “&lt;cls&gt;”**],
que codifica a informação tanto da premissa quanto da hipótese,
(**em três saídas de inferência de linguagem natural**):
implicação, contradição e neutralidade.


In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self, bert):
        super(BERTClassifier, self).__init__()
        self.encoder = bert.encoder
        self.hidden = bert.hidden
        self.output = nn.LazyLinear(3)

    def forward(self, inputs):
        tokens_X, segments_X, valid_lens_x = inputs
        encoded_X = self.encoder(tokens_X, segments_X, valid_lens_x)
        return self.output(self.hidden(encoded_X[:, 0, :]))

A seguir,
o modelo BERT pré-treinado `bert` é alimentado na instância `BERTClassifier` `net` para
a aplicação downstream.
Em implementações comuns de ajuste fino de BERT,
somente os parâmetros da camada de saída do MLP adicional (`net.output`) serão aprendidos do zero.
Todos os parâmetros do codificador BERT pré-treinado (`net.encoder`) e da camada oculta do MLP adicional (`net.hidden`) serão ajustados.


In [ ]:
net = BERTClassifier(bert)

Lembre-se de que
em :numref:`sec_bert`
tanto a classe `MaskLM` quanto a classe `NextSentencePred`
têm parâmetros em seus MLPs empregados.
Esses parâmetros fazem parte daqueles do modelo BERT pré-treinado
`bert` e, portanto, parte dos parâmetros em `net`.
No entanto, tais parâmetros são apenas para computação
a perda da modelagem da linguagem mascarada
e a próxima frase previsão perda
durante o pré-treinamento.
Essas duas funções de perda são irrelevantes para o ajuste fino de aplicações downstream,
assim os parâmetros dos MLPs empregados em
`MaskLM` e `NextSentencePred` não são atualizados (obsoletos) quando o BERT é ajustado.

Para permitir parâmetros com gradientes obsoletos,
o sinalizador `ignore_stale_grad=True` é definido na função `step` de `d2l.train_batch_ch13`.
Usamos esta função para treinar e avaliar o modelo `net` usando o conjunto de treinamento
(`train_iter`) e o conjunto de testes (`test_iter`) do SNLI.
Devido aos recursos computacionais limitados, [**o treinamento**] e a precisão dos testes
pode ser melhorado ainda mais: deixamos suas discussões nos exercícios.


In [ ]:
lr, num_epochs = 1e-4, 5
trainer = torch.optim.Adam(net.parameters(), lr=lr)
loss = nn.CrossEntropyLoss(reduction='none')
net(next(iter(train_iter))[0])
d2l.train_ch13(net, train_iter, test_iter, loss, trainer, num_epochs, devices)

## Resumo

* Podemos ajustar o modelo BERT pré-treinado para aplicações posteriores, como inferência de linguagem natural no conjunto de dados SNLI.
* Durante o ajuste fino, o modelo BERT se torna parte do modelo para o aplicativo downstream. Parâmetros que são relacionados apenas à perda de pré-treinamento não serão atualizados durante o ajuste fino.



## Exercícios

1. Ajuste fino de um modelo BERT pré-treinado muito maior que seja quase tão grande quanto o modelo base BERT original se seu recurso computacional permitir. Defina argumentos na função `load_pretrained_model` como: substituindo 'bert.small' por 'bert.base', aumentando os valores de `num_hiddens=256`, `ffn_num_hiddens=512`, `num_heads=4` e `num_blks=2` para 768, 3072, 12 e 12, respectivamente. Ao aumentar as épocas de ajuste fino (e possivelmente ajustar outros hiperparâmetros), você pode obter uma precisão de teste maior que 0,86?
1. Como truncar um par de sequências de acordo com sua razão de comprimento? Compare este método de truncamento de pares e o usado na classe `SNLIBERTDataset`. Quais são seus prós e contras?


[Discussões](https://discuss.d2l.ai/t/1526)
